In [1]:
import sympy as sym
from sympy.abc import x, y, z
from sympy import Poly
from sympy import LT, LM, LC
from sympy import lcm
from sympy import reduced

#WE CAN CHECK myGroebner WITH sym.groebner(f, x, y, z, order = "lex"), USE SUITABLE GENS AND ORDER

In [2]:
def myDiv(lst, f, t, gen, dom):
    a = [0] * len(lst)
    r = 0
    p = f
    while p != 0:
        i = 0
        divisionoccured = False
        while i<len(lst) and divisionoccured == False:
            if sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[1] == 0 :
                a[i] = a[i] + sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[0]
                p = p - sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[0] * lst[i]
                divisionoccured = True
            else:
                i = i + 1
        if divisionoccured == False:
            r = r + LT(p, order = t)
            p = p - LT(p, order = t)
    return a, r

In [3]:
def calS(f, g, t):
    x_gamma = lcm(LM(f, order = t), LM(g, order = t))
    S = x_gamma/LT(f, order = t) * f - x_gamma/LT(g, order = t) * g
    return S

In [4]:
def myGroebner(f, t, gen, dom):
    g = f.copy()
    rolling = True
    while rolling == True:
        g_control = g.copy()
        for i in range (len(g_control)-1):
            for j in range (i+1, len(g_control)):
                S = calS(g[i], g[j], t = t)
                rem = myDiv(g_control, S, t = t, gen = gen, dom = dom)[1]
                if rem !=0:
                    g.append(Poly(rem, gen, domain = dom))
        if g == g_control:
            rolling = False

    for i in range (len(g)):
        print(f"g{i} = {sym.latex(g[i].as_expr())}")
        
    return g

In [5]:
def myGroebnerCheck(g, t, gen, dom):
    dict_of_S={}
    for i in range(len(g) - 1):
        for j in range(i+1, len(g)):
            S = calS(g[i], g[j], t = t)      
            dict_of_S[i,j] = S

    for item in dict_of_S:
        print(f"S(g{item[0]}, g{item[1]}) = {sym.latex(dict_of_S[item].as_expr())}")
        print(f"Reminder of division with g = {myDiv(g, dict_of_S[item], t = t, gen = gen, dom = dom)[1]}")
        
    return

In [6]:
def myRedGroeb(f, t, gen, dom):
    #HAVING GROEBNER BASIS
    g = myGroebner(f, t = t, gen = gen, dom = dom)

    #HAVING MONIC GROEBNER BASIS
    for i in range(len(g)):
        g[i] = sym.monic(g[i])

    #MAKING LIST OF LT(g):
    lstLT = []
    for i in range (len(g)):
        lstLT.append(Poly(LT(g[i], order = t), gen, domain = dom))

    #CHANGING THE NON REDUCED BASIS INTO ZERO
    h = g.copy()
    for i in range (len(g)):
        while h[i] != 0 :
            denum = lstLT[0:i] + lstLT[i+1:]
            rem = myDiv(denum, Poly(LT(h[i], order = t), gen, domain = dom), t = t, gen = gen, dom = dom)[1]
            if rem == 0 :
                g[i] = 0
                break
            else:
                h[i] = h[i] - LT(h[i], order = t)

    #REMOVING THE ZERO BASIS
    g = list( filter( lambda a: a != 0, g ) )
    print(f"Reduced Groebner = {g}")
    return g

In [18]:
#01

f0 = Poly(-x**3 + y, x, y, z, domain = 'QQ')
f1 = Poly(x**2*y - z, x, y, z, domain = 'QQ')

f = [f0, f1]
print("---reduced Groebner---")
G = myRedGroeb(f, t = "lex", gen =(x,y,z), dom='QQ')

f = [f0, f1]
print("\n---with sympy---")
realG = sym.groebner(f, x, y, z, order = "lex")
print(realG)

question = Poly(x*y**3 - z**2 + y**5 - z**3, x, y, z, domain="QQ")
sym.reduced(question, realG)

---reduced Groebner---
g0 = - x^{3} + y
g1 = x^{2} y - z
g2 = x z - y^{2}
g3 = x y^{3} - z^{2}
g4 = - y^{5} + z^{3}
Reduced Groebner = [Poly(x**3 - y, x, y, z, domain='QQ'), Poly(x**2*y - z, x, y, z, domain='QQ'), Poly(x*z - y**2, x, y, z, domain='QQ'), Poly(x*y**3 - z**2, x, y, z, domain='QQ'), Poly(y**5 - z**3, x, y, z, domain='QQ')]

---with sympy---
GroebnerBasis([x**3 - y, x**2*y - z, x*y**3 - z**2, x*z - y**2, y**5 - z**3], x, y, z, domain='QQ', order='lex')


([Poly(0, x, y, z, domain='ZZ'),
  Poly(0, x, y, z, domain='ZZ'),
  Poly(1, x, y, z, domain='ZZ'),
  Poly(0, x, y, z, domain='ZZ'),
  Poly(1, x, y, z, domain='ZZ')],
 Poly(0, x, y, z, domain='ZZ'))

In [22]:
# 02

f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
print("---reduced Groebner---")
G = myRedGroeb(f, t = "lex", gen =(x,y,z), dom='QQ')

f = [f0, f1, f2]
print("\n---with sympy---")
realG = sym.groebner(f, x, y, z, order = "lex")
print(realG)

question = Poly(x**3*z - 2*y**2, x, y, z, domain="QQ")
sym.reduced(question, realG)

---reduced Groebner---
g0 = x z - y
g1 = x y + 2 z^{2}
g2 = y - z
g3 = - 2 z^{3} - z^{2}
g4 = 2 z^{2} + z
Reduced Groebner = [Poly(y - z, x, y, z, domain='QQ'), Poly(z**2 + 1/2*z, x, y, z, domain='QQ')]

---with sympy---
GroebnerBasis([x*z - z, y - z, 2*z**2 + z], x, y, z, domain='ZZ', order='lex')


([Poly(x**2 + x + 1, x, y, z, domain='ZZ'),
  Poly(-2*y - 2*z, x, y, z, domain='ZZ'),
  Poly(-1, x, y, z, domain='ZZ')],
 Poly(2*z, x, y, z, domain='ZZ'))

In [16]:
#03

f0 = Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain = 'QQ')
f1 = Poly(x**2 + y**2 + z**2 - 2*x, x, y, z, domain = 'QQ')
f2 = Poly(2*x - 3*y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
print("---reduced Groebner---")
myRedGroeb(f, t="lex", gen=(x,y,z), dom="QQ")

f = [f0, f1, f2]
print("\n---with sympy---")
realG = sym.groebner(f, x, y, z, order = "lex")
print(realG)

---reduced Groebner---
g0 = x^{2} + y^{2} + z^{2} - 1
g1 = x^{2} - 2 x + y^{2} + z^{2}
g2 = 2 x - 3 y - z
g3 = 3 y + z - 1
g4 = \frac{13 y^{2}}{4} + \frac{3 y z}{2} + \frac{5 z^{2}}{4} - 1
g5 = \frac{13 y^{2}}{4} + \frac{3 y z}{2} - 3 y + \frac{5 z^{2}}{4} - z
g6 = \frac{10 z^{2}}{9} - \frac{2 z}{9} - \frac{23}{36}
g7 = \frac{400 z^{4}}{1053} - \frac{160 z^{3}}{1053} - \frac{178 z^{2}}{351} + \frac{110 z}{1053} + \frac{184}{1053}
g8 = \frac{400 z^{4}}{1053} - \frac{160 z^{3}}{1053} - \frac{178 z^{2}}{351} + \frac{110 z}{1053} + \frac{184}{1053}
g9 = \frac{10 z^{2}}{9} - \frac{2 z}{9} - \frac{23}{36}
g10 = \frac{400 z^{4}}{1053} - \frac{160 z^{3}}{1053} - \frac{178 z^{2}}{351} + \frac{110 z}{1053} + \frac{184}{1053}
g11 = \frac{400 z^{4}}{1053} - \frac{160 z^{3}}{1053} - \frac{178 z^{2}}{351} + \frac{110 z}{1053} + \frac{184}{1053}
g12 = - \frac{20 z^{2}}{117} + \frac{4 z}{117} + \frac{23}{234}
g13 = - \frac{20 z^{2}}{117} + \frac{4 z}{117} + \frac{23}{234}
g14 = - \frac{40 z^{2}}{117} 

In [31]:
#04

f0 = Poly(x**2*y - z**3, x, y, z, domain="QQ")
f1 = Poly(2*x*y - 4*z - 1, x, y, z, domain="QQ")
f2 = Poly(z - y**2, x, y, z, domain="QQ")
f3 = Poly(x**3 -4*z*y, x, y, z, domain="QQ")

f = [f0, f1, f2, f3]
#print("---reduced Groebner---")
#myRedGroeb(f, t = "lex", gen = (x,y,z), dom = "QQ")

print("\n---with sympy---")
realG = sym.groebner(f, x, y, z, order = "lex")
print(realG)


---with sympy---
GroebnerBasis([1], x, y, z, domain='ZZ', order='lex')
